<a href="https://colab.research.google.com/github/hwan1111/Data-Analysis/blob/main/finance-project/factor-analysis/notebooks/01_factor_data_collection_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 국내외 주식시장 팩터 분석 프로젝트

최근 투자 업계에서는 데이터 기반의 팩터 전략이 주목받고 있다. 글로벌 자산운용사들은 팩터 이론을 활용한 ETF 및 액티브 펀드를 지속적으로 출시하고 있으며, 개인 투자자들도 팩터 기반 투자 전략에 많은 관심을 보이고 있다.\
\
본 프로젝트에서는 yfinance API, pykrx API를 활용해 한국 및 미국 시장에 상장된 종목들의 시세 및 재무 데이터를 수집하고, 대표적인 팩터(가치, 모멘텀, 퀄리티, 로우볼, 사이즈)를 기준으로 데이터를 분석 및 시각화한다.\
데이터는 다음과 같이 수집된다.
- **한국 시장**: `pykrx`를 통해 KOSPI/KOSDAQ 종목 가격 및 투자지표(PER, PBR 등)를 수집
- **미국 시장**: `yfinance`를 통해 미국상장 종목 일별 가격 및 재무 데이터 수집

각 팩터는 다음과 같은 기준으로 정의된다:
- **가치(Value)**: PBR 또는 PER 기준 하위 종목
- **모멘텀(Momentum)**: 최근 3-6개월 수익률 상위 종목
- **퀄리티(Quality)**: ROE 및 안정적 이익 성장률 기반 종목
- **로우볼(Low Volatility)**: 변동성이 낮은 종목
- **사이즈(Size)**: 시가총액이 작은 종목

이 프로젝트의 주요 목표는 다음과 같다.
- yfinance 및 pykrx를 활용한 종목별 데이터 자동 수집
- 시세 및 재무데이터 전처리를 포함한 빅데이터 처리 역량 강화
- 팩터별 종목 분류 및 그룹 수익률 분석
- 시각화를 통한 팩터의 성과 차이를 직관적으로 파악
- 투자 전략 수립을 위한 기초 자료 제공


> 본 프로젝트는 실시간 데이터 분석보다는, 일정 기간의 히스토리컬 데이터를 기준으로 팩터별 특성과 수익룰을 비교하는 데 중점을 두었습니다. 향후 웹소켓 방식을 채택해 실시간성을 추가해볼 예정입니다.

![팩터투자 개념 이미지](https://www.kcie.or.kr/webbook_img?file=webbook/MjAxODEyMjBfOSAg/MDAxNTQ1Mjg3MDY2NDk3.ExJspUcIBudCjG--rYpNVnrKcqn20ei1lykK6P8skaEg.VSIsM1CYg7XiQoz_nFXQSCKkpAurvcAnky4GcG5EOsog.PNG/2.png)

[이미지 출처: 요즘 떠오르는 팩터투자&스마트베타ETF 쉽게 알아가기](https://www.kcie.or.kr/mobile/guide/3/18/web_view?series_idx=&content_idx=387)

# 데이터 수집 및 전처리

이 프로젝트에서는 pykrx API, yfinance API를 통해 한국/미국 주식의 일별 시세와 주요 재무 지표를 수집한다. 수집된 데이터는 향후 팩터 정의 및 백테스트에 활용된다.

## 0. 필요라이브러리 설치 및 마운트

In [ ]:
!pip3 install pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 12.6 MB/s eta 0:00:00


In [ ]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from tqdm import tqdm

from pykrx import stock as pkstock
from pykrx import bond as pkbond
import yfinance

## 1 종목 리스트 수집

In [ ]:
market = 'KOSPI'

tickers = pkstock.get_market_ticker_list(market=market)

tickers

['095570',
 '006840',
 '027410',
 '282330',
 '138930',
 '001460',
 '001465',
 '001040',
 '079160',
 '00104K',
 '000120',
 '011150',
 '011155',
 '001045',
 '097950',
 '097955',
 '000480',
 '000590',
 '012030',
 '005830',
 '016610',
 '000990',
 '000300',
 '001530',
 '015590',
 '000210',
 '000215',
 '375500',
 '37550L',
 '37550K',
 '007340',
 '004840',
 '155660',
 '069730',
 '017860',
 '092780',
 '017940',
 '365550',
 '383220',
 '007700',
 '114090',
 '078930',
 '006360',
 '001250',
 '007070',
 '078935',
 '499790',
 '012630',
 '039570',
 '089470',
 '294870',
 '009540',
 '267250',
 '267270',
 '443060',
 '071970',
 '010620',
 '322000',
 '042670',
 '267260',
 '329180',
 '097230',
 '014790',
 '003580',
 '204320',
 '060980',
 '011200',
 '035000',
 '002460',
 '487570',
 '298050',
 '003560',
 '015360',
 '175330',
 '234080',
 '001060',
 '001067',
 '001065',
 '096760',
 '105560',
 '415640',
 '432320',
 '002380',
 '344820',
 '009070',
 '009440',
 '119650',
 '092220',
 '003620',
 '016380',
 '001390',

In [ ]:
len(tickers)

962

2025년 7월 1일자로 코스피에 총 962개의 종목이 상장되어있다.

### 데이터 저장 및 구조화 전략

팩터 분석에는 종목 수와 분석 기간 모두 많은 데이터를 필요하므로, 시계열 데이터 저장 방식에 따라 성능과 효율성이 크게 달라진다.\
본 프로젝트에서는 다음과 같은 점을 고려해 데이터를 **long format**으로 정규화하였으며, 이를 `.parquet`포맷으로 저장해 효율적인 로딩과 분석이 가능하도록 설계하였다.

- 정규화된 테이블 구조는 그룹 분석, 결측 처리, 수익률 계산 등 다양한 연산에 적합
- Parquet 포맷은 용량을 줄이고 불필요한 컬럼 로딩을 줄여 분석 속도를 향상시킴
- 필요시 DuckDB나 SQLite를 통한 쿼리 기반 분석 확장 가능성 고려

데이터 저장 시 `date`와 `ticker`를 복합 기본키(PK)로 설정함으로써, 각 종목의 일별 데이터를 유일하게 식별하고 분석 효율성을 높였다. 이러한 구조는 시계열 분석, 팩터 계산, 그룹별 수익률 분석 등 다양한 분석 작업에서 높은 일관성과 확장성을 제공할 수 있다.



## 2. 가격 데이터 추출


pykrx의 `get_market_ohlcv()` 함수는 두 가지 방식(`by_date`, `by_ticker`)으로 데이터를 제공하며, 각 방식마다 포함된 컬럼에 차이가 있다.  
본 프로젝트에서는 일별 시계열 분석이 중심이므로 `get_market_ohlcv_by_date()`를 기반으로 데이터를 수집하되, 등락률은 종가 기준 수익률로 계산하여 대체하였다. 이는 수집 속도를 높이면서도 팩터 분석에 필요한 정보는 충분히 확보할 수 있는 실용적인 전략이다.

```
# 1. 날짜 설정
end = datetime.today()
start = end - timedelta(days=365)
start_str = start.strftime('%Y%m%d')
end_str = end.strftime('%Y%m%d')

# 2. 종목 리스트
tickers = pkstock.get_market_ticker_list(market="KOSPI")

# 3. 결과 저장 리스트
all_data = []

# 4. 종목별 수집
for ticker in tqdm(tickers, desc="종목별 수집"):
    try:
        df = pkstock.get_market_ohlcv_by_date(start_str, end_str, ticker).reset_index()
        df['종목코드'] = ticker
        df['종목명'] = pkstock.get_market_ticker_name(ticker)

        # 등락률 계산: 하루 전 대비 종가 수익률
        df['등락률'] = df['종가'].pct_change() * 100  # 백분율 (%)

        # 필요한 컬럼 정리
        df = df[[
            '날짜',
            '종목코드',
            '종목명',
            '종가',
            '등락률'
        ]]
        all_data.append(df)

    except Exception as e:
        print(f"[{ticker}] 에러 발생: {e}")

# 5. 하나로 합치기
df_all = pd.concat(all_data, ignore_index=True)
df_all['날짜'] = pd.to_datetime(df_all['날짜'])

# 6. 저장
df_all.to_parquet("kospi_ohlcv_with_return.parquet")
# 또는 CSV 저장
# df_all.to_csv("kospi_ohlcv_with_return.csv", index=False)

print(f"\n 저장 완료! 총 {len(df_all):,} rows")
```


#### update_kospi_ohlcv(filepath="kospi_ohlcv_with_return.parquet")

In [ ]:
def update_kospi_ohlcv(filepath="kospi_ohlcv_with_return.parquet"):
    today = datetime.today().date()
    yesterday = today - timedelta(days=1)
    yesterday_str = yesterday.strftime('%Y%m%d')

    # 기존 데이터 로드
    if os.path.exists(filepath):
        df_all = pd.read_parquet(filepath)
        print(f"OHLCV 기존 파일 로드됨: {len(df_all):,} rows")

        latest_date = df_all['날짜'].max().date()
        if latest_date >= yesterday:
            print("이미 최신 데이터까지 포함되어 있음.")
            return df_all
        start = latest_date + timedelta(days=1)
    else:
        df_all = pd.DataFrame()
        start = today - timedelta(days=365)

    start_str = start.strftime('%Y%m%d')

    # 종목 리스트
    tickers = pkstock.get_market_ticker_list(market="KOSPI")
    all_data = []

    for ticker in tqdm(tickers, desc=f"종목별 OHLCV 수집"):
        try:
            df = pkstock.get_market_ohlcv_by_date(start_str, yesterday_str, ticker).reset_index()
            if df.empty:
                continue

            df['종목코드'] = ticker
            df['종목명'] = pkstock.get_market_ticker_name(ticker)

            df = df[[
                '날짜',
                '종목코드',
                '종목명',
                '종가'
            ]]

            all_data.append(df)
        except Exception as e:
            print(f"[{ticker}] 에러 발생: {e}")

    if not all_data:
        print("수집된 데이터 없음.")
        return df_all

    df_new = pd.concat(all_data, ignore_index=True)
    df_new['날짜'] = pd.to_datetime(df_new['날짜'])

    # 기존과 병합 후 중복 제거
    df_all = pd.concat([df_all, df_new], ignore_index=True)
    df_all.drop_duplicates(subset=['날짜', '종목코드'], keep='last', inplace=True)

    # 등락률 재계산
    df_all.sort_values(['종목코드', '날짜'], inplace=True)
    df_all['등락률'] = df_all.groupby('종목코드')['종가'].pct_change() * 100

    df_all.sort_index(inplace=True)

    # 저장
    df_all.to_parquet(filepath)
    print(f"{yesterday}까지 반영 완료. 총 {len(df_all):,} rows 저장됨.")

    return df_all

본 프로젝트에서는 종목과 날짜 정보를 기반으로 정규화된 long-format 테이블을 구축했다. 이는 하나의 통합된 구조에서 다양한 분석 목적에 유연하게 대응할 수 있도록 설계한 것으로, 데이터 저장 방식부터 모델 학습 구조까지 일관성을 확보하는 데 중점을 두었다.

특히 long-format 구조는 머신러닝 기반 분류/회귀 모델과 시계열 딥러닝 모델 모두에 적합한 전처리 기반을 제공한다. 예를 들어, 전 종목 데이터를 통합한 상태에서는 feature engineering과 벡터화된 입력 구성이 용이하며, 종목별 시계열 데이터를 분리하면 순차 입력 기반의 딥러닝 모델에 효과적으로 활용할 수 있다.

또한, 데이터 저장 및 조회의 효율성을 고려해 SQL 기반 접근을 병행했다. 리밸런싱 시점 분석과 같이 특정 날짜에 전체 종목을 조회하는 경우에는 날짜 기준인덱싱을, 개별 종목의 시계열 특성을 분석하거나 입력 데이터로 사용할 경우에는 티커 기준 인덱싱을 활용함으로써 쿼리 성능과 분석 유연석을 동시에 확보했다.

## 3. 재무재표 데이터 추출

```
# 1. 날짜 설정
end = datetime.today()
start = end - timedelta(days=365)
start_str = start.strftime('%Y%m%d')
end_str = end.strftime('%Y%m%d')

# 2. 종목 리스트
tickers = pkstock.get_market_ticker_list(market="KOSPI")

# 3. 결과 저장 리스트
all_data = []

# 4. 종목별 수집
for ticker in tqdm(tickers, desc="종목별 수집"):
    try:
        df = pkstock.get_market_fundamental_by_date(start_str, end_str, ticker).reset_index()
        df['종목코드'] = ticker
        df['종목명'] = pkstock.get_market_ticker_name(ticker)

        # 필요한 컬럼 정리
        df = df[[
            '날짜',
            '종목코드',
            '종목명',
            'BPS',
            'PER',
            'PBR',
            'EPS',
            'DIV',
            'DPS'
        ]]
        all_data.append(df)

    except Exception as e:
        print(f"[{ticker}] 에러 발생: {e}")

# 5. 하나로 합치기
df_all = pd.concat(all_data, ignore_index=True)
df_all['날짜'] = pd.to_datetime(df_all['날짜'])

# 6. 저장
df_all.to_parquet("kospi_fundamental_with_return.parquet")
# 또는 CSV 저장
# df_all.to_csv("kospi_fundamental_with_return.csv", index=False)

print(f"\n 저장 완료! 총 {len(df_all):,} rows")
```

```
종목별 수집:   4%|▍         | 38/962 [01:06<21:03,  1.37s/it][365550] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:   8%|▊         | 81/962 [02:22<19:35,  1.33s/it][415640] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:   9%|▊         | 82/962 [02:23<15:15,  1.04s/it][432320] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  14%|█▎        | 131/962 [03:42<16:11,  1.17s/it][400760] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  14%|█▍        | 134/962 [03:46<14:05,  1.02s/it][338100] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  17%|█▋        | 165/962 [04:34<15:27,  1.16s/it][395400] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  38%|███▊      | 370/962 [09:59<10:51,  1.10s/it][377190] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  39%|███▉      | 376/962 [10:07<10:59,  1.12s/it][330590] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  41%|████      | 390/962 [10:26<10:24,  1.09s/it][357430] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  41%|████      | 392/962 [10:28<08:48,  1.08it/s][088980] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  41%|████      | 393/962 [10:28<06:55,  1.37it/s][094800] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  43%|████▎     | 409/962 [10:54<12:36,  1.37s/it][396690] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  43%|████▎     | 410/962 [10:55<09:41,  1.05s/it][357250] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  46%|████▋     | 447/962 [11:49<09:24,  1.10s/it][448730] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  56%|█████▌    | 536/962 [14:07<08:24,  1.18s/it][204210] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  58%|█████▊    | 557/962 [14:38<07:16,  1.08s/it][481850] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  58%|█████▊    | 558/962 [14:38<05:38,  1.19it/s][404990] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  58%|█████▊    | 559/962 [14:39<04:47,  1.40it/s][293940] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  62%|██████▏   | 596/962 [15:41<07:18,  1.20s/it][140910] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  63%|██████▎   | 607/962 [15:57<07:34,  1.28s/it][900140] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  69%|██████▊   | 660/962 [17:30<07:45,  1.54s/it][088260] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  70%|██████▉   | 670/962 [17:46<06:00,  1.23s/it][350520] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  70%|██████▉   | 671/962 [17:46<04:35,  1.05it/s][334890] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  73%|███████▎  | 699/962 [18:29<05:19,  1.22s/it][348950] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  77%|███████▋  | 744/962 [19:39<05:22,  1.48s/it][145270] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  77%|███████▋  | 745/962 [19:39<04:02,  1.12s/it][417310] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  78%|███████▊  | 746/962 [19:39<03:21,  1.07it/s][357120] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  84%|████████▍ | 812/962 [21:18<03:01,  1.21s/it][950210] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  86%|████████▌ | 823/962 [21:34<02:37,  1.13s/it][152550] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집:  93%|█████████▎| 895/962 [23:24<01:21,  1.22s/it][451800] 에러 발생: "None of [Index(['날짜', 'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS'], dtype='object')] are in the [columns]"
종목별 수집: 100%|██████████| 962/962 [25:13<00:00,  1.57s/it]
 저장 완료! 총 224,401 rows
```

#### update_kospi_fundamental(filepath='kospi_fundamental_with_return.parquet', json_path='excluded_tickers.json')

In [ ]:
EXCLUDED_TICKERS = {
    '365550', '415640', '432320', '400760', '338100',
    '395400', '377190', '330590', '357430', '088980',
    '094800', '396690', '357250', '448730', '204210',
    '481850', '404990', '293940', '140910', '900140',
    '088260', '350520', '334890', '348950', '145270',
    '417310', '357120', '950210', '152550', '451800'
}

In [ ]:
def update_kospi_fundamental(
        filepath='kospi_fundamental_with_return.parquet',
        json_path='excluded_tickers.json'):

    with open(json_path, 'r') as file:
        EXCLUDED_TICKERS = set(json.load(file)['excluded_tickers'])

    today = datetime.today().date()
    yesterday = today - timedelta(days=1)
    yesterday_str = yesterday.strftime('%Y%m%d')

    # 기존 데이터 로드
    if os.path.exists(filepath):
        df_all = pd.read_parquet(filepath)
        print(f"Fundamental 기존 파일 로드됨: {len(df_all):,} rows")

        latest_date = df_all['날짜'].max().date()
        if latest_date >= yesterday:
            print('이미 최신 데이터까지 포함되어 있음.')
            return df_all
        start = latest_date + timedelta(days=1)
    else:
        df_all = pd.DataFrame()
        start = today - timedelta(days=365)

    start_str = start.strftime('%Y%m%d')

    # 종목 리스트
    tickers = pkstock.get_market_ticker_list(market='KOSPI')
    all_data = []

    for ticker in tqdm(tickers, desc=f'종목별 재무재표 수집'):
        try:
            if ticker in EXCLUDED_TICKERS:
                continue

            df = pkstock.get_market_fundamental_by_date(start_str, yesterday_str, ticker).reset_index()
            if df.empty:
                continue

            df['종목코드'] = ticker
            df['종목명'] = pkstock.get_market_ticker_name(ticker)

            df = df[[
                '날짜',
                '종목코드',
                '종목명',
                'BPS',
                'PER',
                'PBR',
                'EPS',
                'DIV',
                'DPS'
            ]]
            all_data.append(df)
        except Exception as e:
            print(f'[{ticker}] 에러 발생: {e}')

    if not all_data:
        print('수집된 데이터 없음.')
        return df_all

    df_new = pd.concat(all_data, ignore_index=True)
    df_new['날짜'] = pd.to_datetime(df_new['날짜'])

    # 병합 및 저장
    df_all = pd.concat([df_all, df_new], ignore_index=True)
    df_all.drop_duplicates(subset=['날짜', '종목코드'], keep='last', inplace=True)
    df_all.to_parquet(filepath)

    print(f"{yesterday_str}까지 반영 완료. 총 {len(df_all):,} rows → {filepath}")
    return df_all

재무지표 수집 과정에서 리츠(REITs), 우선주, 스팩(SPAC), 지분증권 등 일반 상장 기업과 다른 회계 구조를 가지거나, 재무지표가 공시되지 않는 비표준 종목들이 다수 존재하였다. 이러한 종목은 PER, PBR, EPS 등의 주요 팩터 산출 지표가 존재하지 않거나 의미가 없기 때문에, 본 프로젝트에서는 해당 종목들을 사전 필터링을 통해 분석 대상에서 제외하였다.

에러를 유발하는 비표준 종목 목록은 JSON 파일 형태로 별도로 관리하여 코드와 데이터 정의를 분리하였다. 이를 통해 코드 수정 없이 제외 종목을 관리할 수 있으며, 다른 분석 스크립트에서도 동일한 기준을 재사용할 수 있도록 하여 유지보수성과 일관성을 높였다.

## 4. 시가총액 데이터 추출

```
# 1. 날짜 설정
end = datetime.today()
start = end - timedelta(days=365)
start_str = start.strftime('%Y%m%d')
end_str = end.strftime('%Y%m%d')

# 2. 종목 리스트
tickers = pkstock.get_market_ticker_list(market="KOSPI")

# 3. 결과 저장 리스트
all_data = []

# 4. 종목별 수집
for ticker in tqdm(tickers, desc="종목별 수집"):
    try:
        df = pkstock.get_market_cap_by_date(start_str, end_str, ticker).reset_index()
        df['종목코드'] = ticker
        df['종목명'] = pkstock.get_market_ticker_name(ticker)

        # 필요한 컬럼 정리
        df = df[[
            '날짜',
            '종목코드',
            '종목명',
            '시가총액',
            '거래량',
            '거래대금'
        ]]
        all_data.append(df)

    except Exception as e:
        print(f"[{ticker}] 에러 발생: {e}")

# 5. 하나로 합치기
df_all = pd.concat(all_data, ignore_index=True)
df_all['날짜'] = pd.to_datetime(df_all['날짜'])

# 6. 저장
df_all.to_parquet("kospi_marketcap_with_return.parquet")
# 또는 CSV 저장
# df_all.to_csv("kospi_ohlcv_with_return.csv", index=False)

print(f"\n 저장 완료! 총 {len(df_all):,} rows")
```

#### update_kospi_marketcap(filepath="kospi_marketcap_with_return.parquet")

In [ ]:
def update_kospi_marketcap(filepath="kospi_marketcap_with_return.parquet"):
    today = datetime.today().date()
    yesterday = today - timedelta(days=1)
    yesterday_str = yesterday.strftime('%Y%m%d')

    # 기존 데이터 로드
    if os.path.exists(filepath):
        df_all = pd.read_parquet(filepath)
        print(f'MarketCap 기존 파일 로드됨: {len(df_all)}: rows')

        latest_date = df_all['날짜'].max().date()
        if latest_date >= yesterday:
            print('이미 최신 데이터까지 포함되어 있음')
            return df_all
        start = latest_date + timedelta(days=1)
    else:
        df_all = pd.DataFrame()
        start = today - timedelta(days=365)

    start_str = start.strftime('%Y%m%d')

    # 종목 리스트
    tickers = pkstock.get_market_ticker_list(market='KOSPI')
    all_data = []

    for ticker in tqdm(tickers, desc='종목별 시가총액 수집'):
        try:
            df = pkstock.get_market_cap_by_date(start_str, yesterday_str, ticker).reset_index()
            if df.empty:
                continue

            df['종목코드'] = ticker
            df['종목명'] = pkstock.get_market_ticker_name(ticker)

            df = df[[
                '날짜',
                '종목코드',
                '종목명',
                '시가총액',
                '거래량',
                '거래대금'
            ]]

            all_data.append(df)
        except Exception as e:
            print(f'[{ticker}] 에러 발생: {e}')

    if not all_data:
        print('수집된 데이터 없음.')
        return df_all

    df_new = pd.concat(all_data, ignore_index=True)
    df_new['날짜'] = pd.to_datetime(df_new['날짜'])

    # 병합 및 저장
    df_all = pd.concat([df_all, df_new], ignore_index=True)
    df_all.drop_duplicates(subset=['날짜', '종목코드'], keep='last', inplace=True)
    df_all.to_parquet(filepath)

    print(f"{yesterday_str}까지 반영 완료. 총 {len(df_all):,} rows → {filepath}")
    return df_all

## 5. 업종(섹터) 데이터 추출

업종(섹터) 데이터는 `pykrx`에서는 추출하지 못하는 점 때문에 본 프로젝트에서는 KRX 정보데이터시스템의 '업종분류 현황' 데이터를 활용해 각 종목의 업종(섹터) 정보를 매핑하였다. 해당 파일에는 티커별로 속한 업종명이 포함되어 있다.

업종 정보는 향후 다음과 같은 분석에 활용할 수 있다.
- **섹터 내 정규화**: 재무지표를 업종 내 상대적 기준으로 스케일링 할 수 있음
- **섹터 기반 팩터 분석**: 업종 특성에 따라 팩터 효과가 다르게 나타나는지를 평가
- **포트폴리오 분산 투자 전략**: 업종 분산을 고려한 자산 배분 가능

In [ ]:
def update_kospi_sector(csv_path='업종분류 현황.csv'):
    df_sector = pd.read_csv(csv_path, encoding='euc-kr')
    df_sector['종목코드'] = df_sector['종목코드'].astype(str).str.zfill(6)
    return df_sector

## 6. 통합 데이터프레임 구성

#### update_kospi()

In [ ]:
def update_kospi():
    price_df = update_kospi_ohlcv()
    fundamental_df = update_kospi_fundamental()
    marketcap_df = update_kospi_marketcap()
    sector_df = update_kospi_sector()

    merge_df = pd.merge(
        price_df,
        sector_df[['종목코드', '업종명']],
        on=['종목코드'],
        how='left'
    )

    merge_df = pd.merge(
        merge_df,
        fundamental_df.drop(columns=['종목명']),
        on=['날짜', '종목코드'],
        how='left'
    )

    merge_df = pd.merge(
        merge_df,
        marketcap_df.drop(columns=['종목명']),
        on=['날짜', '종목코드'],
        how='left'
    )

    cols = ['날짜', '종목코드', '종목명', '업종명', '종가',
            '등락률', '시가총액', '거래량', '거래대금',
            'BPS', 'PER', 'PBR', 'EPS', 'DIV', 'DPS']

    return merge_df[cols]

In [ ]:
df = update_kospi()
df.head()

OHLCV 기존 파일 로드됨: 230,611 rows


종목별 OHLCV 수집: 100%|██████████| 962/962 [04:19<00:00,  3.71it/s]


2025-07-03까지 반영 완료. 총 231,573 rows 저장됨.
Fundamental 기존 파일 로드됨: 223,481 rows


종목별 재무재표 수집: 100%|██████████| 962/962 [02:53<00:00,  5.54it/s]


20250703까지 반영 완료. 총 224,413 rows → kospi_fundamental_with_return.parquet
MarketCap 기존 파일 로드됨: 230611: rows


종목별 시가총액 수집: 100%|██████████| 962/962 [02:51<00:00,  5.60it/s]


20250703까지 반영 완료. 총 231,573 rows → kospi_marketcap_with_return.parquet


,날짜,종목코드,종목명,업종명,종가,등락률,시가총액,거래량,거래대금,BPS,PER,PBR,EPS,DIV,DPS
0,2024-07-03,095570,AJ네트웍스,일반서비스,4325,NaN,195718182675,172857,751087580,9326.0,11.78,0.46,367.0,6.24,270.0
1,2024-07-04,095570,AJ네트웍스,일반서비스,4355,0.693642,197075765445,88181,380902455,9326.0,11.87,0.47,367.0,6.20,270.0
2,2024-07-05,095570,AJ네트웍스,일반서비스,4375,0.459242,197980820625,109637,483212730,9326.0,11.92,0.47,367.0,6.17,270.0
3,2024-07-08,095570,AJ네트웍스,일반서비스,4465,2.057143,202053568935,104014,463397405,9326.0,12.17,0.48,367.0,6.05,270.0
4,2024-07-09,095570,AJ네트웍스,일반서비스,4440,-0.559910,200922249960,78035,346864710,9326.0,12.10,0.48,367.0,6.08,270.0


## 7. 결측치 처리 및 전처리

### 결측치 처리

통합된 시세, 재무재표, 시가총액 데이터프레임에서 결측치가 많은 종목을 식별하고, 이후 분석 신뢰도를 높이기 위한 사전 정제 작업을 수행했다.

- ticker 기준으로 그룹화해 각 열의 결측치 개수를 집계
- 종목별 총 결측치 수를 기준으로 결측치 비중이 큰 종목을 파악
- 단일 팩터 분석 시 해당 팩터에 한정해 결측치를 고려할 수 있도록 설계

In [ ]:
na_count = df.groupby('종목코드').apply(lambda g: g.isna().sum(), include_groups=False)

na_count['총결측치'] = na_count.sum(axis=1)
na_count_sorted = na_count.sort_values('총결측치', ascending=False)

na_count_sorted.head(40)

,날짜,종목명,업종명,종가,등락률,시가총액,거래량,거래대금,BPS,PER,PBR,EPS,DIV,DPS,총결측치
종목코드,,,,,,,,,,,,,,,
432320,0,0,0,0,1,0,0,0,241,241,241,241,241,241,1447
417310,0,0,0,0,1,0,0,0,241,241,241,241,241,241,1447
404990,0,0,0,0,1,0,0,0,241,241,241,241,241,241,1447
400760,0,0,0,0,1,0,0,0,241,241,241,241,241,241,1447
396690,0,0,0,0,1,0,0,0,241,241,241,241,241,241,1447
395400,0,0,0,0,1,0,0,0,241,241,241,241,241,241,1447
377190,0,0,0,0,1,0,0,0,241,241,241,241,241,241,1447
365550,0,0,0,0,1,0,0,0,241,241,241,241,241,241,1447
357430,0,0,0,0,1,0,0,0,241,241,241,241,241,241,1447


In [ ]:
def drop_nan(df, excluded_tickers_path='excluded_tickers.json'):
    with open(excluded_tickers_path) as file:
        excluded_tickers = set(json.load(file)['excluded_tickers'])

    df = df[~df['종목코드'].isin(excluded_tickers)].reset_index(drop=True)
    df['등락률'] = df['등락률'].fillna(0)
    return df

df = drop_nan(df)
df

,날짜,종목코드,종목명,업종명,종가,등락률,시가총액,거래량,거래대금,BPS,PER,PBR,EPS,DIV,DPS
0,2024-07-03,095570,AJ네트웍스,일반서비스,4325,0.000000,195718182675,172857,751087580,9326.0,11.78,0.46,367.0,6.24,270.0
1,2024-07-04,095570,AJ네트웍스,일반서비스,4355,0.693642,197075765445,88181,380902455,9326.0,11.87,0.47,367.0,6.20,270.0
2,2024-07-05,095570,AJ네트웍스,일반서비스,4375,0.459242,197980820625,109637,483212730,9326.0,11.92,0.47,367.0,6.17,270.0
3,2024-07-08,095570,AJ네트웍스,일반서비스,4465,2.057143,202053568935,104014,463397405,9326.0,12.17,0.48,367.0,6.05,270.0
4,2024-07-09,095570,AJ네트웍스,일반서비스,4440,-0.559910,200922249960,78035,346864710,9326.0,12.10,0.48,367.0,6.08,270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224408,2025-07-03,079980,휴비스,화학,3200,2.564103,110400000000,64037,203442395,7140.0,0.00,0.45,0.0,0.00,0.0
224409,2025-07-03,005010,휴스틸,금속,4830,3.870968,271388402250,747096,3561002332,19927.0,12.17,0.24,397.0,3.11,150.0
224410,2025-07-03,000540,흥국화재,보험,4280,-1.947308,274958520600,141196,611375110,11857.0,2.93,0.36,1459.0,0.00,0.0
224411,2025-07-03,000545,흥국화재우,보험,7300,2.384292,5606400000,10985,80755200,0.0,0.00,0.00,0.0,0.00,0.0


통합된 데이터프레임의 결측치를 분석한 결과, 대부분은 스팩(SPAC), 리츠(REITs) 등 특수 증권에 대한 재무지표 누락과 각 티커별 시계열에서 첫 번째 관측일의 등락률 결측으로 나타났다. 특수 증권은 재무 데이터 수집의 한계로 인해 분석 대상에서 제외하였고, 첫 번째 등락률 결측은 계산 불가능성에 기인한 것으로 판단되어 0으로 대체하였다. 이는 실제 수익률이 존재하지 않는 시점에서 발생한 결측으로, 정보 왜곡 없이 분석 및 모델 학습에 포함시키기에 적절하다고 판단하였다.

### 정규화

- 종가
    - 전처리 방식: MinMaxScaler
    - 그룹 기준: 종목코드
    - 이유: 주식의 가격 수준은 종목마다 크게 다르며, 절대적인 가격 자체는 모델에 큰 의미가 없고 변동성 또는 패턴이 더 중요하다고 생각했다. 따라서 종목별로 최저가최고가 구간을 0-1사이로 정규화함으로써 각 종목의 상대적인 가격 흐름에 집중할 수 있도록 하였다. MinMaxScaler는 시계열 분석 시 원래의 형태를 유지하면서 범위를 조정하는 데 유리해 선택하였다.

- 시가총액
    - 전처리 방식: log변환 후 MinMaxScaler
    - 그룹 기준: 전체 혹은 섹터
    - 이유: 시가총액은 종목 간 편차가 매우 크며, 대형주의 비중이 과도하게 반영될 수 있다. 로그 변환을 통해 분포를 안정화하고, 정규화를 통해 모델이 규모 차이에 과도하게 영향을 받지 않도록 조정했다.

- 거래량, 거래대금
    - 전처리 방식: log1p 변환 후 RobustScaler
    - 그룹 기준: 종목코드
    - 이유: 거래량과 거래대금은 극단적인 이상치를 자주 포함하는 특성이다. 로그 변환으로 스케일을 줄이고, 중앙값 기반의 RobustScaler를 적용해 이상치의 영향을 최소화하였다.

- 재무 지표(PER, EPS)
    - 전처리 방식: Z-Score 표준화 또는 RobustScaler
    - 그룹 기준: 섹터
    - 이유: 두 지표 모두 이익을 기반한 수치로, 음수일 수 있으며, 이상치의 분포가 심하다(예: 적자기업). PER는 주가 대비 수익률을 EPS는 주당순이익을 의미하지만 둘 다 수익성 관련 지표이므로 함께 처리할 수 있다. 평균과 표준편차 기반의 Z-Score를 사용하여 스케일을 맞추되, 이상치가 많은 경우에는 중앙값 기반의 RobustScaler를 사용해 안정적으로 정규화하였다.

- 재무 지표(PBR)
    - 전처리 방식: RobustScaler
    - 그룹 기준: 종목 코드
    - 이유: PBR은 주가 대비 자산가치를 나타내는 지표로, 업종에 따라 평균 수준이 매우 다르며 극단적인 고평가/저평가 구간이 자주 발생한다. 평균보다는 중앙값 기준 정규화가 더 적절하다고 판단해 RobustScaler를 적용하였다.

- 재무 지표(BPS)
    - 전처리 방식: Z-Score 표준화
    - 그룹 기준: 종목 코드
    - 이유: BPS(주당순자산)는 절대값이 크고, 규모가 큰 기업일수록 수치가 크다. EPS와는 달리 음수보다는 편차가 큰 양의 수가 대부분이므로, 이상치가 적고 Z-Score 표준화로도 안정적으로 정규화 가능.

- 재무 지표(DIV, DPS)
    - 전처리 방식: MinMaxScaler
    - 그룹 기준: 전체 혹은 섹터
    - 이유: 배당지표들은 대부분 0에 가깝고, 특정 종목만 값이 존재하는 경우가 많다. 분포가 매우 치우쳐 있어 로그 변환보다는 0-1 사이로 스케일을 맞추는 방식이 더 적합하다고 판단. DIV(배당수익률), DPS(주당배당금)는 보조지표로 사용되며, 모델에 직접적으로 큰 영향을 미치지 않도록 범위만 조정하였다.
    

In [ ]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

In [ ]:
def make_scaled_df(df):
    # 종가: 종목코드별 MinMax 정규화
    df['종가_scaled'] = df.groupby('종목코드')['종가'].transform(
        lambda x: MinMaxScaler().fit_transform(x.values.reshape(-1, 1)).flatten()
    )

    # 시가총액: 업종명별 로그 + MinMax 정규화
    df['시가총액_sector_scaled'] = df.groupby('업종명')['시가총액'].transform(
        lambda x: MinMaxScaler().fit_transform(np.log1p(x.values).reshape(-1, 1)).flatten()
    )

    # 시가총액: 전체 로그 + MinMax 정규화
    df['시가총액_scaled'] = MinMaxScaler().fit_transform(
        np.log1p(df['시가총액']).values.reshape(-1, 1)
    ).flatten()

    # 거래량: 전체 로그 + Robust 정규화
    df['거래량_scaled'] = RobustScaler().fit_transform(
        np.log1p(df['거래량']).values.reshape(-1, 1)
    ).flatten()

    # 거래대금: 종목코드별 로그 + MinMax 정규화
    df['거래대금_scaled'] = df.groupby('종목코드')['거래대금'].transform(
        lambda x: MinMaxScaler().fit_transform(np.log1p(x.values).reshape(-1, 1)).flatten()
    )

    # PER: 업종명별 Robust 정규화
    df['PER_sector_scaled'] = df.groupby('업종명')['PER'].transform(
        lambda x: RobustScaler().fit_transform(x.values.reshape(-1, 1)).flatten()
    )

    # EPS: 업종명별 Robust 정규화
    df['EPS_sector_scaled'] = df.groupby('업종명')['EPS'].transform(
        lambda x: RobustScaler().fit_transform(x.values.reshape(-1, 1)).flatten()
    )

    # PBR: 종목코드별 Robust 정규화
    df['PBR_sector_scaled'] = df.groupby('종목코드')['PBR'].transform(
        lambda x: RobustScaler().fit_transform(x.values.reshape(-1, 1)).flatten()
    )

    # BPS: 종목코드별 Standard 정규화
    df['BPS_sector_scaled'] = df.groupby('종목코드')['BPS'].transform(
        lambda x: StandardScaler().fit_transform(x.values.reshape(-1, 1)).flatten()
    )

    # DIV, DPS: 전체 MinMax 정규화
    df['DIV_scaled'] = MinMaxScaler().fit_transform(df[['DIV']])
    df['DPS_scaled'] = MinMaxScaler().fit_transform(df[['DPS']])

    return df

In [ ]:
scaled_df = make_scaled_df(df)
scaled_df

,날짜,종목코드,종목명,업종명,종가,등락률,시가총액,거래량,거래대금,BPS,...,시가총액_sector_scaled,시가총액_scaled,거래량_scaled,거래대금_scaled,PER_sector_scaled,EPS_sector_scaled,PBR_sector_scaled,BPS_sector_scaled,DIV_scaled,DPS_scaled
0,2024-07-03,095570,AJ네트웍스,일반서비스,4325,0.000000,195718182675,172857,751087580,9326.0,...,0.390958,0.364845,0.384421,0.559773,0.280285,-0.161221,0.000000,-0.451642,0.251918,0.010000
1,2024-07-04,095570,AJ네트웍스,일반서비스,4355,0.693642,197075765445,88181,380902455,9326.0,...,0.392028,0.365401,0.148597,0.366756,0.287411,-0.161221,0.125000,-0.451642,0.250303,0.010000
2,2024-07-05,095570,AJ네트웍스,일반서비스,4375,0.459242,197980820625,109637,483212730,9326.0,...,0.392737,0.365770,0.224901,0.434389,0.291370,-0.161221,0.125000,-0.451642,0.249092,0.010000
3,2024-07-08,095570,AJ네트웍스,일반서비스,4465,2.057143,202053568935,104014,463397405,9326.0,...,0.395888,0.367408,0.206454,0.422486,0.311164,-0.161221,0.250000,-0.451642,0.244247,0.010000
4,2024-07-09,095570,AJ네트웍스,일반서비스,4440,-0.559910,200922249960,78035,346864710,9326.0,...,0.395019,0.366956,0.105770,0.340146,0.305622,-0.161221,0.250000,-0.451642,0.245458,0.010000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224408,2025-07-03,079980,휴비스,화학,3200,2.564103,110400000000,64037,203442395,7140.0,...,0.371866,0.318769,0.036503,0.403109,-0.446988,-0.321146,1.571429,-2.214145,0.000000,0.000000
224409,2025-07-03,005010,휴스틸,금속,4830,3.870968,271388402250,747096,3561002332,19927.0,...,0.463755,0.391149,0.897269,0.379451,0.869231,0.075666,0.000000,2.214145,0.125555,0.005556
224410,2025-07-03,000540,흥국화재,보험,4280,-1.947308,274958520600,141196,611375110,11857.0,...,0.494234,0.392201,0.313535,0.488378,-0.093151,-0.014573,5.000000,-2.214145,0.000000,0.000000
224411,2025-07-03,000545,흥국화재우,보험,7300,2.384292,5606400000,10985,80755200,0.0,...,0.061457,0.078946,-0.581152,0.740320,-0.895890,-0.233774,0.000000,0.000000,0.000000,0.000000
